In [39]:
import pandas as pd
import parser
import imp

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
imp.reload(parser)

<module 'parser' from '/home/tonko22/PycharmProjects/npo_classification/parser.py'>

In [10]:
!git commit -a -m "minor fix"

[master acbfa2d] minor fix
 3 files changed, 1503 insertions(+), 91 deletions(-)
 rename Data_cleaning.ipynb => Data_cleaning_text_extraction.ipynb (99%)
 create mode 100644 wmd.ipynb


In [13]:
dataset = pd.read_excel("np_ngo_classification.xlsx")
dataset.head(1)

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Ссылка на соц сеть,Вид деятельности 1,...,Вид деятельности 3,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12
0,Москва,"Межрегиональная общественная благотворительная организация ""Общество защиты прав потребителей и ...",5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-й км, д 6",0,0,"http://greenstartup.ru/, http:/9911850.ru/",https://princip1.livejournal.com/,3_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Drop empty rows
sites = dataset[~dataset["Ссылка на сайт"].isnull()]
sites.drop(columns=["Ссылка на соц сеть"], inplace=True)
social = dataset[~dataset["Ссылка на соц сеть"].isnull()]

# Merging social web-page to a web-site, adding new data samples 
social["Ссылка на сайт"] = social["Ссылка на соц сеть"]
social.drop(columns=["Ссылка на соц сеть"], inplace=True)
concat_dataset = pd.concat([sites, social], ignore_index=True)

# Fixing mistakes
concat_dataset["Ссылка на сайт"].loc[62] = "http://арктика-надежда.рф" 
concat_dataset["Ссылка на сайт"].loc[154] = "http://9911850.ru/"

/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versu

In [23]:
def link_splitter(url_string):
    result = []
    urls = []
    
    for url in url_string.split(","):
        urls.a"бфурал.рф"ppend(url)
        
    for url in urls:
        for each in url.split("\n"):
            result.append(each)
            
    return result

concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(link_splitter)
dataset.dropna(axis=0, subset=["Ссылка на сайт"], inplace=True)
concat_dataset["num_sites"] = concat_dataset["Ссылка на сайт"].apply(len)

In [26]:
# ! WARNING: This cell needs to be executed 3 times!
# TODO: 3 times-loop
max_len = len(concat_dataset)
for index, row in concat_dataset.iterrows():
    if len(row["Ссылка на сайт"])>1:
        sites = row["Ссылка на сайт"]
        last_site = sites.pop(-1)
        concat_dataset["Ссылка на сайт"].loc[index] = sites
        row["Ссылка на сайт"] = [last_site]
        concat_dataset.loc[max_len] = row
        max_len += 1

/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [45]:
import requests
from bs4 import BeautifulSoup
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("http://greenstartup.ru/", timeout=(5.05, 27), headers=headers)
html = r.text
soup = BeautifulSoup(html)
data = soup.findAll(text=True)
data_filtered = [each.strip() for each in data if each!="\n" and each!=" "]
data_filtered = [each for each in data_filtered if each!=""]
data_filtered

["function set_cookie(){var now = new Date();var time = now.getTime();time += 19360000 * 1000;now.setTime(time);document.cookie='beget=begetok'+'; expires='+now.toGMTString()+'; path=/';}set_cookie();location.reload();;"]

In [27]:
concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(lambda lst: lst[0].strip())

In [41]:
concat_dataset["web_text"] = concat_dataset["Ссылка на сайт"].apply(parser.get_text_contents)

/home/tonko22/PycharmProjects/npo_classification/parser.py:45: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 45 of the file /home/tonko22/PycharmProjects/npo_classification/parser.py. To get rid of this warning, pass the additional argument 'features="html5lib"' to the BeautifulSoup constructor.

  data = soup.findAll(text=True)
/home/tonko22/PycharmProjects/npo_classification/parser.py:57: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this

ConnectTimeout: HTTPConnectionPool(host='xn----7sbhgebudrdu3cu5p.xn--p1ai', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f9bf554e0b8>, 'Connection to xn----7sbhgebudrdu3cu5p.xn--p1ai timed out. (connect timeout=5.05)'))

In [20]:
concat_dataset.dropna(subset=["web_text"]).to_pickle(path="np_ngo_classification_links.pkl")

In [19]:
concat_dataset.to_pickle(path="np_ngo_classification_links.pkl")

In [20]:
concat_dataset

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Вид деятельности 1,Вид деятельности 2,Вид деятельности 3,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12
0,Москва,"Межрегиональная общественная благотворительная организация ""Общество защиты прав потребителей и ...",5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-й км, д 6",0,0,"http://greenstartup.ru/, http:/9911850.ru/",3_2,6_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Москва,"Межрегиональная благотворительная общественная организация инвалидов ""Общество пациентов с перви...",7702370774,2007-08-28,"г Москва, Мещанский р-н, ул Трифоновская, д 49, оф 41",1,1,http://oppid.ru/,2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Москва,"Некоммерческая организация Благотворительный фонд ""Столица Милосердия""",7702472455,2014-02-11,"г Москва, Мещанский р-н, ул Щепкина, д 9 стр 1",1,1,capital-mercy.ru,1_9,1_2,1_7,2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Москва,"Российский общественный благотворительный фонд ветеранов (пенсионеров) войны, труда и Вооруженны...",7704058560,1998-06-05,"г Москва, р-н Соколиная Гора, ул Щербаковская, д 50/52",1,1,http://www.rbfondveteranov.ru/,1_4,5_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Москва,"Благотворительный фонд поддержки и реализации программ в сфере культуры ""АВС""",7704278848,2011-09-30,"г Москва, р-н Хамовники, пер Знаменский Б., д 13 стр 2",0,0,http://www.avccharity.com/,4_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Москва,"Благотворительный фонд ""Фонд Валерия Гергиева""",7709402778,2003-01-27,"г Москва, р-н Арбат, ул Молчановка Б., д 23 к 2",0,0,http://gergiev-charity.com/rus/,4_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Москва,Международный Благотворительный Фонд Владимира Спивакова,7710012370,1994-05-20,"г Москва, Пресненский р-н, Шведский тупик, д 3, оф 17",1,0,http://spivakov.ru/fund/contacts/,4_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Москва,"Благотворительный фонд поддержки детей с особенностями развития ""Я есть!""",7710479817,2012-05-02,"г Москва, Пресненский р-н, пер Кондратьевский Б., д 4 к 3, оф 29",1,1,http://yaest.ru/,1_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Москва,"Благотворительный фонд помощи бездомным животным ""Ковчег""",7716451494,2014-11-14,"г Москва, Ярославский р-н, Ярославское шоссе, д 2 к 1, кв 157",1,0,http://fond-kovcheg.ru,6_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Москва,Благотворительный фонд поддержки детей имени императрицы Александры Федоровны Романовой,7701064809,2011-03-22,"105082, ГОРОД МОСКВА, УЛИЦА ФРИДРИХА ЭНГЕЛЬСА, 75, СТР. 3, ОФ. № 1",1,1,http://www.vfonde.ru/,1_7,1_2,2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
concat_dataset

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Вид деятельности 1,Вид деятельности 2,...,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12,num_sites,web_text
0,Москва,"Межрегиональная общественная благотворительная организация ""Общество защиты прав потребителей и ...",5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-й км, д 6",0,0,http://greenstartup.ru/,3_2,6_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,
1,Москва,"Межрегиональная благотворительная общественная организация инвалидов ""Общество пациентов с перви...",7702370774,2007-08-28,"г Москва, Мещанский р-н, ул Трифоновская, д 49, оф 41",1,1,http://oppid.ru/,2_1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,ОППИД | Общество пациентов с первичным иммунодефицитом Главная Информационный канал Контакты Вы ...
2,Москва,"Некоммерческая организация Благотворительный фонд ""Столица Милосердия""",7702472455,2014-02-11,"г Москва, Мещанский р-н, ул Щепкина, д 9 стр 1",1,1,http://capital-mercy.ru,1_9,1_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None
3,Москва,"Российский общественный благотворительный фонд ветеранов (пенсионеров) войны, труда и Вооруженны...",7704058560,1998-06-05,"г Москва, р-н Соколиная Гора, ул Щербаковская, д 50/52",1,1,http://www.rbfondveteranov.ru/,1_4,5_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"105187, Москва, Щербаковская ул., 50/52 Главная Персоналии Российскому фонду ветеранов 25 лет Ак..."
4,Москва,"Благотворительный фонд поддержки и реализации программ в сфере культуры ""АВС""",7704278848,2011-09-30,"г Москва, р-н Хамовники, пер Знаменский Б., д 13 стр 2",0,0,http://www.avccharity.com/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Перейти к основному содержанию О фонде Проекты Выставки Музыка Контакты Фонд AVC Charity выступи...
5,Москва,"Благотворительный фонд ""Фонд Валерия Гергиева""",7709402778,2003-01-27,"г Москва, р-н Арбат, ул Молчановка Б., д 23 к 2",0,0,http://gergiev-charity.com/rus/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"Благотворительный фонд «Фонд Валерия Гергиева» <!--\r\nvar dp=""../"";\r\n\r\nvar title = ""Благотв..."
6,Москва,Международный Благотворительный Фонд Владимира Спивакова,7710012370,1994-05-20,"г Москва, Пресненский р-н, Шведский тупик, д 3, оф 17",1,0,http://spivakov.ru/fund/contacts/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Контактная информация и реквизиты | Фонд Спивакова О фонде Сделать пожертвование Заявки Галереи ...
7,Москва,"Благотворительный фонд поддержки детей с особенностями развития ""Я есть!""",7710479817,2012-05-02,"г Москва, Пресненский р-н, пер Кондратьевский Б., д 4 к 3, оф 29",1,1,http://yaest.ru/,1_7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None
8,Москва,"Благотворительный фонд помощи бездомным животным ""Ковчег""",7716451494,2014-11-14,"г Москва, Ярославский р-н, Ярославское шоссе, д 2 к 1, кв 157",1,0,http://fond-kovcheg.ru,6_4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Фонд «Ковчег» | Поможем вместе Вход Как помочь? Новости Животные Проекты Истории Как помочь? Пож...
9,Москва,Благотворительный фонд поддержки детей имени императрицы Александры Федоровны Романовой,7701064809,2011-03-22,"105082, ГОРОД МОСКВА, УЛИЦА ФРИДРИХА ЭНГЕЛЬСА, 75, СТР. 3, ОФ. № 1",1,1,http://www.vfonde.ru/,1_7,1_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,


In [219]:
concat_dataset["web_text"].loc[22]

'105187, Москва, Щербаковская ул., 50/52 Главная Персоналии Российскому фонду ветеранов 25 лет Акции Российского фонда ветеранов Постановления о материальной помощи в 2017году. Контакты «Туризм для граждан старшего поколения» По направлению Российского фонда ветеранов за поступившие средства от спонсоров в Герантологическом центре «Монино» прошли курс реабилитации ветераны и участники ВОВ Новости В ВОРОНЕЖЕ ВОЕННЫЕ ПИСАТЕЛИ ПОДВЕЛИ ИТОГИ 2018 ГОДА Заседание Воронежской областной общественной организации «Союз военных писателей» (СВП) прошло 3 ноября 2018 года. На нём были подведены итоги работы в 2018 году. ИЗВЕСТНОЙ ВОРОНЕЖСКОЙ ПОЭТЕССЕ – 70 ЛЕТ Сотрудники регионального отделения Российского фонда ветеранов совместно с членами Воронежской областной общественной организации «Союз военных писателей» чествовали ветерана труда, известную поэтессу. «РУСИЧИ-ЦЕНТР» выдал Российскому фонду ветеранов Золотой сертификат благотворителя, свидетельствующий о том, что Фонд принял в 2018 году активн

In [6]:
mapping = pd.read_excel("np_ngo_mappings.xlsx")
mapping

In [7]:
dataset["links"] = dataset["Ссылка на сайт"].apply(parser.get_links)

/home/ubuntu/PycharmProjects/nko_classification/parser.py:16: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 16 of the file /home/ubuntu/PycharmProjects/nko_classification/parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


In [60]:
tr = cyrtranslit.to_cyrillic('http://oppid.ru/zasedanie-rabochey-gruppyi-po-redkim-zabolevaniyam-pod-predsedatelstvom-valentinyi-ivanovnyi-shirokovoy/', 'ru')

In [53]:
nlp = Russian()
# nlp = spacy.load('xx_ent_wiki_sm')
tokenizer = Tokenizer(nlp.vocab)

In [62]:
doc = nlp(tr)
print([token.text for token in doc])

['хттп://оппид.ру', '/', 'заседание', '-', 'рабочей', '-', 'группыи', '-', 'по', '-', 'редким', '-', 'заболеваниям', '-', 'под', '-', 'председателством', '-', 'валентиныи', '-', 'ивановныи', '-', 'широковой/']
